## Things to do:
#### 1. Read train.csv 
    Seperate train.csv into training data (data entries that have scores) and testing data (entries without scores, will also submit these predictions)
    
#### 2. seperate training data into y and x

# Think of your model
1. Linear Regression
2. Sentiment analysis on Summary + Text, maybe even get emotions
3. make product ID categorical?
4. 
    

In [40]:
import csv
import csv
import numpy as np
import statsmodels.api as sm
'''from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions, SentimentOptions'''
from textblob import TextBlob as TB
    
def ReadData(filename):
    f = open(filename, "r")
    reader = csv.reader(f)
    header = next(reader)
    i = 0
    data = list(reader)
    train_data = []
    test_data = []
    for i in range(len(data)):
        if data[i][4] == "":
            test_data += [data[i]]
        else:
            train_data += [data[i]]
    return train_data, test_data, data      

In [41]:
filename = "train.csv"
x = ReadData(filename)
train = x[0]
test = x[1]
data = x[2]

In [55]:
len(train)

460804

In [5]:
def Products(data):
    dic = {}
    for entry in data:
        if entry[3] not in dic:
            dic[entry[3]] = 1
        else:
            dic[entry[3]] += 1
    return dic

In [34]:
def FeatureVec(data):
    '''
    Output: adds a couple for columns to every entry
    Think of what to add:
        1. helpfulness numerator / helpfulness denominator 
        2. Sentiment of text data
        3. Get emotions of text data?
    '''
    
    for i in range(len(data)):
        
        # append helpfulness numerator / helpfulness denominator
        if int(data[i][0]) == 0:
            data[i] += [0]
        else:
            data[i].append(float(data[i][1]) / float(data[i][0]))
        temp = data[i][5] + " " + data[i][6]
        temp = temp.strip()
        textblob = TB(temp)
        data[i] += [textblob.sentiment.polarity, textblob.sentiment.subjectivity]
            
        '''
        Attempted to use Watson NLU to get sentiment scores and emotions of every summary + text
        Unfortunately, I only have 30, 000 requests per month so this won't work.
    
        
        response = nlu.analyze(
                    text = temp,
                    #html = article_html,
                    features=Features(
                        entities=EntitiesOptions(
                          emotion=True,
                          sentiment=True,
                          limit=2),

                        sentiment=SentOpt
                    ))
        # Use Watson Natural Language Understanding to get the emotions of the summary + text 
        # Also gets an overall sentiment score and positive/negative label
        # add all these to our feature vectors
        #print(temp)
        #print(response)
        if len(response['entities']) == 0:
            sent_score = response['sentiment']['document']['score']
            if response['sentiment']['document']['label'] == "positive":
                label = 1
            else:
                label = 0
            anger = sent_score
            disgust = sent_score
            fear = sent_score
            joy = sent_score
            sadness = sent_score
        else:
            #print(response)
            #print("-------------")
            if 'emotion' not in response["entities"][0]:
                sent = response["sentiment"]["document"]
                if sent['label'] == "positive":
                    label = 1
                else:
                    label = 0
                sent_score = sent["score"]
                anger = sent_score
                disgust = sent_score
                fear = sent_score
                joy = sent_score
                sadness = sent_score
            else:
                emotions = response['entities'][0]['emotion']
                anger = emotions['anger']
                disgust = emotions['disgust']
                fear = emotions["fear"]
                joy = emotions["joy"]
                sadness = emotions["sadness"]
                sent = response['entities'][0]['sentiment']
                if sent['label'] == 'positive':
                    label = 1
                else:
                    label = 0
                sent_score = sent['score']
        data[i] += [anger, disgust, fear, joy, sadness, label, sent_score]'''
    for entry in data:
        if entry[4] == "":
            entry[4] = 0
    y_var = [int(entry[4]) for entry in data]
    feature_vec = [ [int(entry[0]),int(entry[1])] + entry[-3:] for entry in data]
    return y_var, feature_vec, data
    

In [47]:
X = FeatureVec(data)

In [48]:
# writes the augmented dataset to features.csv
Write(X[2])

In [53]:
len(X[0])

560804

In [37]:
x_var = X[1]
y_var = X[0]
X_col = sm.add_constant(x_var)
model = sm.OLS(y_var, X_col)

In [49]:
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                 4.039e+04
Date:                Sat, 05 May 2018   Prob (F-statistic):               0.00
Time:                        14:53:54   Log-Likelihood:            -6.9484e+05
No. Observations:              460804   AIC:                         1.390e+06
Df Residuals:                  460798   BIC:                         1.390e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6244      0.007    516.504      0.000       3.611       3.638
x1            -0.1524      0.001   -175.213      0.000      -0.154      -0.151
x2             0.1584      0.001    167.601      0.000       0.157       0.160
x3             0.0849      0.004     23.550      0.000       0.078       0.092
x4             2.7351      0.007    367.932      0.000       2.721       2.750
x5            -0.2548      0.012    -20.631      0.000      -0.279      -0.231
==============================================================================
Omnibus:                    58908.722   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           111019.023
Skew:                          -0.829   Prob(JB):                         0.00
Kurtosis:                       4.742   Cond. No.                         102.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
results.params

array([ 3.62438627, -0.15238043,  0.15838217,  0.08489057,  2.73511017,
       -0.25475554])

In [13]:
X_test = FeatureVec(test)

In [16]:
X_test_col = sm.add_constant(X_test[1])

In [17]:
predictions = results.predict(X_test_col)

In [18]:
int_pred = [int(round(x)) for x in predictions]

In [28]:
def Text(data):
    # Outputs a list of (summary + text)
    return [entry[5] + " " + entry[6] for entry in data]

In [58]:
len(data)

560804

In [19]:
def Write(data):
    # Writes data to a csv file
    with open("features.csv", "w") as f:
        writer = csv.writer(f)
        header = ["HelpfulnessDenominator","HelpfulnessNumerator","Id","ProductId","Score",
          "Summary", "Text", "Time", "UserId", "HelpfulnessRatio", "SentimentPolarity", 
         "SentimentSubjectivity"]
        writer.writerow(header)
        writer.writerows(data)

In [24]:
def Results(int_predictions, test):
    # get final prediction
    results = []
    for i in range(len(int_predictions)):
        results += [[test[i][2], int_predictions[i]]]
    with open("kaggle_results.csv", "w") as f:
        writer = csv.writer(f)
        header = ["Id", "Score"]
        writer.writerow(header)
        writer.writerows(results)
    return results
    

In [25]:
x = Results(int_pred, test)

In [63]:
import random
def KfoldTest(data):
    # Program that calculates the average MSE of model.fit() with 5-fold cross validation
    # param data: data entries with their additional features added already
    
    # Holds MSE values from every fold
    Predictions = []
    
    for i in range(5):
        
        training_data = []
        testing_data = []
        # Create random sample of 400 training data, 100 teting data
        training_index = random.sample(range(460804), 368000)
        for data_index in range(460804):
            if data_index in training_index:
                training_data += [data[data_index]]
            else:
                testing_data += [data[data_index]]
        
        # Get dependent variable y
        y_var = [int(entry[4]) for entry in training_data]
        
        # Get independent variable x
        x_var = [[int(entry[0]),int(entry[1])] + entry[-3:] for entry in training_data]
        
        X = sm.add_constant(x_var)
        model = sm.OLS(y_var, X)
        
        # Create regular model and regularized model
        results = model.fit()
        
        # Get testing data dependent variable
        testing_scores = [int(entry[4]) for entry in testingData]

        # Get testing data independent variable
        testing_X = [[int(entry[0]),int(entry[1])] + entry[-3:] for entry in data]            
        testing_X = sm.add_constant(testing_X)
        
        # Make predictions with testing data according to model above
        predictions = results.predict(testing_X)
        
        # Calculate MSEs
        MSE = mean_squared_error(predictions, testing_scores)
        
        Predictions += [MSE]
        
    # Return average MSE over each fold
    return sum(Predictions) / len(Predictions)

In [ ]:
x = KfoldTest(train)